# Pré-processamento - DB_gender

## Algoritmos para correlação de atributos

### Importando bibliotecas e módulos para o pré-processamento

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder

### Importando arquivo como variável array

In [ ]:
df_gender = pd.read_csv("XXXX")
np_gender = df_gender.to_numpy()

### Análise Exploratória de Dados (EDA)

In [ ]:
#Observando os dados
print(df_gender.shape, "\n")
print(df_gender.head(1), "\n")
#print(df_gender.columns, "\n")
print(df_gender.describe(), "\n")

In [ ]:
#Plotando graficamente os dados
df_gender.iloc[:, 0:12].hist(bins=50, figsize=(12,8))
#Para ajustar os títulos e rótulos de cada eixo (se houver)
for ax in plt.gcf().axes:
    ax.tick_params(axis='x', labelsize=8)
    ax.tick_params(axis='y', labelsize=8)
    ax.title.set_fontsize(8)

plt.tight_layout()
plt.show()

#### Algoritmos de correlação de atributos

- Pearson Correlation (atributos numéricos):  

    Encontra o relacionamento linear dos atributos ideal para classificação binária (0_1, A_B).

In [ ]:
#Calculando correlação de Pearson entre cada atributo e a classe alvo ('gender')
correlations = df_gender.corr(method='pearson')['gender'].drop('gender')

#Top 9 maiores correlações positivas e negativas
top9_pos = correlations.sort_values(key=abs, ascending=False).head(9)
#top9_neg = correlations.sort_values(key=abs, ascending=True).head(9)

print("Correlação de Pearson entre atributos e classe alvo:")
print("Top 9 atributos mais positivamente correlacionados com 'gender':", '\n', top9_pos, '\n',)
#print("Top 9 atributos mais negativamente correlacionados com 'gender':", '\n', top9_neg)

#Selecionando os atributos mais correlacionados com classe alvo ('gender')
att_pearson_corr = top9_pos.index.tolist() #+ top9_neg.index.tolist()
print(att_pearson_corr)

#===============================================
#Plotando as 9 maiores correlações positivas
plt.figure(figsize=(6,3))
top9_pos.plot(kind='barh', color='green', alpha=0.7, label="Positiva")

#Plotando as 9 maiores correlações negativas
#top9_neg.plot(kind='barh', color='red', alpha=0.7, label="Negativa")

plt.title("Top atributos correlacionados", fontsize=14)
plt.ylabel("Coeficiente de Correlação (Pearson)")
plt.tight_layout()
plt.show()

- Random Forest (atributos numéricos):  

    Encontra relações complexas e não-lineares entre o astributos

In [ ]:
#Preparação dos dados (é necessário para o algoritmo Random Forest)
X = df_gender.drop(columns=['gender'])      #Apenas atributos sem classe alvo
y = df_gender['gender']                     #Classe alvo

#Divisão dos dados em conjuntos de treinamento e teste
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

#======================
#Modelo Random Forest
rf = RandomForestClassifier(
    n_estimators=235,        #número de árvores
    max_depth=20,            #profundidade ilimitada
    random_state=42,
    n_jobs=-1
)
rf.fit(X_train, y_train)

#Ordenação dos atributos pela importância
importances = pd.Series(rf.feature_importances_, index=X.columns)
importances_sorted = importances.sort_values(ascending=False)

print("\nImportância dos atributos segundo o Random Forest:")
print(importances_sorted.head(9))  #mostra os 9 mais importantes

att_RF_corr = importances_sorted.head(9).index.tolist()
print(att_RF_corr)

plt.figure(figsize=(6,3))
importances_sorted.head(9).plot(kind='barh')
plt.title("Top 9 atributos mais importantes (Random Forest)", fontsize=14)
plt.ylabel("Importância (Gini Importance)")
plt.tight_layout()
plt.show()

#### Atributos com grande correlação com a classe alvo

In [ ]:
best_att = []
for i in range(9):
    for j in range(9):
        if att_pearson_corr[i] == att_RF_corr[j]:
            best_att.append(att_pearson_corr[i])
            

print(best_att)

#### PairPlot da distribuição de gênero pelos melhores atributos

In [ ]:
#Cria um novo DataFrame contendo os melhores atributos e a coluna de gênero
df_plot = df_gender[best_att + ['gender']].copy()

#Renomear a coluna e mapear os valores para Mulher = 0, Homem = 1
df_plot = df_plot.rename(columns={'gender': 'Gênero'})
df_plot['Gênero'] = df_plot['Gênero'].map({0: 'Mulher', 1: 'Homem'})

print(df_plot.head())

#Cria um gráfico de dispersão múltipla entre entre os atributos e a classe alvo
plot = sns.pairplot(df_plot, hue='Gênero', hue_order=['Mulher', 'Homem'], palette='Set1', corner=True, height=1.3, aspect=1.3)
plt.show()

## Importando bibliotecas e módulos para o treinamento de modelos

In [ ]:
#Algoritmos de Machine Learning
from sklearn.tree import DecisionTreeClassifier                      #Árvores de decisão
from sklearn.neighbors import KNeighborsClassifier                   #kNN
from sklearn.naive_bayes import GaussianNB                           #Naive Bayes
from sklearn.svm import LinearSVC                                    #Máquinas de Vetores de Suporte
from sklearn.linear_model import SGDClassifier                       #Gradiente Estocástico
import joblib                                                        #Salvamento e carregamento de modelos

#Utilidades
from sklearn.preprocessing import StandardScaler                     #Normalização de atributos
import time                                                          #Tempo de execução
from pathlib import Path                                             #Caminhos de arquivos

#Avaliação e seleção de modelos
from sklearn.model_selection import (
    cross_val_score,                                                #Validação cruzada simples
    StratifiedKFold,                                                #Validação estratificada
    GridSearchCV,                                                   #Busca em grade de hiperparâmetros
)

#Métricas de avaliação
from sklearn.metrics import (
    accuracy_score, precision_score, recall_score, f1_score,        #Métricas principais
    confusion_matrix, classification_report,                        #Relatórios e matrizes
)

### Preparado conjunto de dados CSI para treinamento dos algoritmos de *Machine Learning*

Total: 4000 amostras de 10 pessoas (400 amostras por pessoa) $\rightarrow$ Conjunto universo (U) = 4000  
Conjunto de treinamento + Conjuto de teste= 3200 (8 pessoas)  
Amostra de validação = 800 (2 pessoas)

In [ ]:
#Construção de dados de treinamento e teste (Foram retirados 2 individuos de cada classe - 20% dos dados)
df_gender = pd.read_csv("XXXX")

#Selecionar apenas os atributos mais correlacionados com a classe alvo ['rpi1_sc121', 'rpi1_sc120', 'rpi1_sc119', 'rpi1_sc122', 'rpi1_sc118']
df_gender_selected = df_gender[best_att + ['gender']]

print(df_gender_selected.head(), '\n')

#Preparação dos dados
X = df_gender_selected.drop(columns=['gender'])      #Apenas atributos sem classe alvo
y = df_gender_selected['gender']                     #Classe alvo

#### Treinamento com validação cruzada

- Padronização dos dados usando Z-score (pelo StandardScaler)
- Seleção dos melhores hiperparâmetros para cada algoritmo (pelo GridSearchCV)
- Treinamento usando a validação cruzada e a seleção do melhor modelo para cada algoritmo (pelo StratifieldKFold)

In [ ]:
#Padronização dos dados CSI com média=0 e desvio_padrão=1 (Z-score)
scaler = StandardScaler()
X_scaled = scaler.fit_transform(X)

#Definindo os modelos e hiperparâmetros
modelos_com_grid = {
    "Decision Tree": (DecisionTreeClassifier(random_state=42), {
        'max_depth': [3, 5, 7, None],
        'criterion': ['gini', 'entropy']
    }),
    "kNN": (KNeighborsClassifier(), {
        'n_neighbors': [3, 5, 7]
    }),
    "Naive Bayes": (GaussianNB(), {}),
    "SVM Linear": (LinearSVC(random_state=42, dual=False, max_iter=10000), {
        'C': [0.1, 1, 10]
    })
}

#Definindo as validações (configuração nested CV)
inner_cv = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)
outer_cv = StratifiedKFold(n_splits=3, shuffle=True, random_state=42)

#Avaliação de modelos
resultados_val = {}
melhor_modelo = None
melhor_nome = ""
melhor_acc = 0

output_dir = Path("results/cross-validation")
output_dir.mkdir(parents=True, exist_ok=True)

relatorio_path = output_dir / "relatorio.txt"

with open(relatorio_path, "w", encoding="utf-8") as f:
    f.write("Relatório da Validação Cruzada:\n\n")

    for nome, (modelo, grid) in modelos_com_grid.items():
        print(f"\nModel: {nome}")

        #Caso haja grid, usar GridSearchCV
        if grid:
            grid_search = GridSearchCV(modelo, grid, cv=inner_cv, scoring='accuracy', n_jobs=-1)
        else:
            grid_search = modelo

        #Validação cruzada
        scores = cross_val_score(grid_search, X=X_scaled, y=y, cv=outer_cv, n_jobs=-1)
        resultados_val[nome] = scores

        #Treino final (melhor modelo) e medição de tempo
        start = time.perf_counter()
        grid_search.fit(X_scaled, y)
        end = time.perf_counter()
        tempo_treino = end - start

        final_model = grid_search.best_estimator_ if hasattr(grid_search, 'best_estimator_') else grid_search

        #Predição e avaliação do melhor modelo
        y_pred = final_model.predict(X_scaled)
        acc = accuracy_score(y, y_pred)

        #Relatório
        relatorio = classification_report(y, y_pred, target_names=["Mulher", "Homem"], digits=4)
        f.write(f"Model: {nome}\n")
        f.write(f"Tempo de construção: {tempo_treino:.6f} segundos\n")
        f.write(relatorio)
        f.write("\n" + "-" * 60 + "\n")

        print(relatorio)
        print(f"Tempo de construção do modelo: {tempo_treino:.6f} segundos")

        #Atualização do melhor modelo
        if acc > melhor_acc:
            melhor_acc = acc
            melhor_modelo = final_model
            melhor_nome = nome

#### Fazendo predições (classificação) com novos dados

- Carrega os novos dados (DB_gender_400_fp_2p.csv);  
- Aplica o mesmo scaler usado no treino;  
- Fazer as predições com o melhor_modelo encontrado anteriormente; e  
- Gerar métricas e plota uma matriz de confusão.

In [ ]:
#Carregar novos dados
novos_dados_path = "XXXX"
df_novos = pd.read_csv(novos_dados_path)

#Selecionar mesmos atributos usados no treino
df_novos_selected = df_novos[best_att + ['gender']]
X_new = df_novos_selected.drop(columns=['gender'])
y_true = df_novos_selected['gender']

#Aplicar o mesmo scaler do treino
X_new_scaled = scaler.transform(X_new)

#Fazer predição com o melhor modelo
y_pred = melhor_modelo.predict(X_new_scaled)

#Relatório de métricas
print("\nRelatório completo:\n")
print(classification_report(y_true, y_pred, target_names=["Mulher", "Homem"], digits=4))

#Matriz de confusão
cm = confusion_matrix(y_true, y_pred)

plt.matshow(cm, cmap=plt.cm.Blues)
plt.title("Matriz de Confusão - Novos Dados")
plt.colorbar()
classes = ["Mulher", "Homem"]
plt.xticks(np.arange(len(classes)), classes)
plt.yticks(np.arange(len(classes)), classes)

#Inserir valores dentro da matriz
for i in range(cm.shape[0]):
    for j in range(cm.shape[1]):
        plt.text(j, i, cm[i, j], ha="center", va="center", color="red", fontsize=12)

plt.xlabel("Predição")
plt.ylabel("Real")
plt.show()
